# Verify All Available Solutions

Let's check the scores of all available solutions to find the best one.

In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit
from numba.typed import List as NumbaList
import glob

# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    angle_rad = angle_deg * math.pi / 180.0
    cos_a, sin_a = math.cos(angle_rad), math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([[0.0, TIP_Y], [TOP_W/2, TIER_1_Y], [TOP_W/4, TIER_1_Y],
                    [MID_W/2, TIER_2_Y], [MID_W/4, TIER_2_Y], [BASE_W/2, BASE_Y],
                    [TRUNK_W/2, BASE_Y], [TRUNK_W/2, TRUNK_BOTTOM_Y],
                    [-TRUNK_W/2, TRUNK_BOTTOM_Y], [-TRUNK_W/2, BASE_Y],
                    [-BASE_W/2, BASE_Y], [-MID_W/4, TIER_2_Y], [-MID_W/2, TIER_2_Y],
                    [-TOP_W/4, TIER_1_Y], [-TOP_W/2, TIER_1_Y]], dtype=np.float64)
    for i in range(15):
        rx = pts[i,0] * cos_a - pts[i,1] * sin_a
        ry = pts[i,0] * sin_a + pts[i,1] * cos_a
        vertices[i,0], vertices[i,1] = rx + cx, ry + cy
    return vertices

@njit(cache=True)
def polygon_bounds(vertices):
    min_x, min_y = vertices[0,0], vertices[0,1]
    max_x, max_y = vertices[0,0], vertices[0,1]
    for i in range(1, vertices.shape[0]):
        x, y = vertices[i,0], vertices[i,1]
        if x < min_x: min_x = x
        if x > max_x: max_x = x
        if y < min_y: min_y = y
        if y > max_y: max_y = y
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def get_side_length(all_vertices):
    min_x, min_y, max_x, max_y = math.inf, math.inf, -math.inf, -math.inf
    for verts in all_vertices:
        x1, y1, x2, y2 = polygon_bounds(verts)
        if x1 < min_x: min_x = x1
        if y1 < min_y: min_y = y1
        if x2 > max_x: max_x = x2
        if y2 > max_y: max_y = y2
    return max(max_x - min_x, max_y - min_y)

@njit(cache=True)
def calculate_score_numba(all_vertices):
    side = get_side_length(all_vertices)
    return side * side / len(all_vertices)

print("Functions compiled")

Functions compiled


In [2]:
def load_and_score(filepath):
    try:
        df = pd.read_csv(filepath)
        if 'id' not in df.columns or 'x' not in df.columns:
            return None, "Invalid format"
        
        all_xs, all_ys, all_degs = [], [], []
        for n in range(1, 201):
            group = df[df["id"].str.startswith(f"{n:03d}_")].sort_values("id")
            if len(group) != n:
                return None, f"Wrong count for N={n}: {len(group)}"
            for _, row in group.iterrows():
                x_val = str(row["x"]).replace('s', '').replace('x', '')
                y_val = str(row["y"]).replace('s', '').replace('y', '')
                deg_val = str(row["deg"]).replace('s', '').replace('d', '')
                all_xs.append(float(x_val))
                all_ys.append(float(y_val))
                all_degs.append(float(deg_val))
        
        # Calculate total score
        total = 0.0
        idx = 0
        for n in range(1, 201):
            vertices = NumbaList()
            for i in range(n):
                vertices.append(get_tree_vertices(all_xs[idx+i], all_ys[idx+i], all_degs[idx+i]))
            total += calculate_score_numba(vertices)
            idx += n
        
        return total, "OK"
    except Exception as e:
        return None, str(e)

print("Scorer ready")

Scorer ready


In [3]:
# Find all CSV files
csv_files = [
    '/home/code/santa-2025-csv/santa-2025.csv',
    '/home/code/telegram/71.97.csv',
    '/home/code/telegram/72.49.csv',
    '/home/code/smartmanoj_submission.csv',
] + glob.glob('/home/code/santa25-public/*.csv')

print(f"Found {len(csv_files)} CSV files")

results = []
for f in csv_files:
    score, status = load_and_score(f)
    results.append((f, score, status))
    if score is not None:
        print(f"{f.split('/')[-1]}: {score:.6f}")
    else:
        print(f"{f.split('/')[-1]}: ERROR - {status}")

Found 20 CSV files


santa-2025.csv: 70.676102


71.97.csv: 71.972027


72.49.csv: 72.495739


smartmanoj_submission.csv: 70.743774


...

santa2025_ver2_v63.csv: 72.947427


santa2025_ver2_v69.csv: 72.850110


submission_JKoT1.csv: 72.489483


submission_opt1.csv: 70.990692


santa2025_ver2_v68.csv: 72.939233


In [4]:
# Sort by score (lower is better)
valid_results = [(f, s, st) for f, s, st in results if s is not None]
valid_results.sort(key=lambda x: x[1])

print("\n" + "="*60)
print("BEST SOLUTIONS (lower is better):")
print("="*60)
for f, score, _ in valid_results[:10]:
    print(f"{score:.6f} - {f.split('/')[-1]}")

print(f"\nTarget: 68.922808")
print(f"Best available: {valid_results[0][1]:.6f}")
print(f"Gap: {valid_results[0][1] - 68.922808:.6f}")


BEST SOLUTIONS (lower is better):
70.676102 - santa-2025.csv
70.743774 - smartmanoj_submission.csv
70.926150 - submission_70_926149550346.csv
70.936674 - submission_70_936673758122.csv
70.990692 - submission_opt1.csv
71.972027 - 71.97.csv
72.489348 - submission_JKoT2.csv
72.489483 - submission_JKoT1.csv
72.489488 - submission_JKoT3.csv
72.489504 - submission_JKoT4.csv

Target: 68.922808
Best available: 70.676102
Gap: 1.753294
